# Simple Moving Average Crossover Backtest Demo

This notebook demonstrates a full backtesting pipeline using the modular framework:
1. Setup Environment
2. Import modules
3. Load price and risk-free rate data
4. Generate trading signals (SMA Crossover for SPY)
5. Run the backtest simulation
6. Display performance metrics
7. Plot equity curve and drawdown periods

### 1. Setup Environment

Add the project's root directory to the Python path. This allows us to import the `src` package which contains our custom modules.

In [ ]:
# Add project root to Python path to import 'src' package
import sys
import pathlib

# Assuming the notebook is in 'quant-backtester/notebooks/'
# project_root will be 'quant-backtester/'
project_root = pathlib.Path("..").resolve()

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Added project root {project_root} to sys.path")
else:
    print(f"Project root {project_root} already in sys.path")

### 2. Imports

Import necessary third-party libraries and our custom modules from the `src` package.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from src import data_loader, strategy, backtester, metrics

### 3. Load Data

Fetch historical price data for SPY (S&P 500 ETF) from 2008 to the present. We also load daily risk-free rates from a local CSV file located in the project root.

In [ ]:
# Define parameters for data loading
tickers = ["SPY"]
start_date = "2008-01-01"
end_date = None  # None means fetch up to the latest available data

# Load historical OHLCV data
print(f"Loading price data for {tickers} from {start_date}...")
prices_df = data_loader.get_prices(tickers, start=start_date, end=end_date, cache=True)

# Extract Adjusted Close prices for SPY as our main price series for the strategy
price = prices_df[tickers[0]]['Adj Close']
print(f"Loaded SPY Adj Close prices: {len(price)} data points from {price.index.min().date()} to {price.index.max().date()}\n")

# Load risk-free rates
# project_root was defined in the first code cell
rf_filename = "risk-free.csv"
rf_path = project_root / rf_filename 
print(f"Loading risk-free rates from {rf_path}...")
rf = data_loader.load_risk_free(path=str(rf_path))
print(f"Loaded risk-free rates: {len(rf)} data points from {rf.index.min().date()} to {rf.index.max().date()}")

### 4. Generate Signals

Generate trading signals using a Simple Moving Average (SMA) Crossover strategy. We'll use a 50-day SMA as the short-term average and a 200-day SMA as the long-term average.

In [ ]:
# Define SMA window lengths
short_window = 50
long_window = 200

print(f"Generating SMA crossover signals (short: {short_window}, long: {long_window})...")
positions = strategy.sma_crossover_signal(price, short=short_window, long=long_window)
print("Signal generation complete.")
print(f"Positions series: {len(positions)} data points.")
print(f"Example positions (last 5 days):\n{positions.tail()}")

### 5. Run Backtest

Simulate the trading strategy using the `Backtester` class. This will apply the generated positions to the historical price data, accounting for transaction costs and incorporating risk-free rates for periods when the strategy is flat.

In [ ]:
# Define transaction cost in basis points (10 bps = 0.1%)
transaction_cost_bps = 10.0

print("Initializing and running the backtest...")
bt_instance = backtester.Backtester(prices=price, positions=positions, rf=rf, cost_bps=transaction_cost_bps)
result = bt_instance.run()
print("Backtest complete.")

### 6. Show Performance Metrics

Display a summary of the strategy's performance, including CAGR, Sharpe Ratio, Max Drawdown, Win Rate, and Turnover.

In [ ]:
print("Calculating and displaying performance metrics...")
performance_summary = metrics.summarize(result)
print("\nPerformance Summary:")
print(performance_summary)

### 7. Plot Results

Visualize the strategy's equity curve over time and the corresponding drawdown periods.

In [ ]:
print("Plotting equity curve and drawdown periods...")
# Create a figure with two subplots (one for equity, one for drawdown)
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(14, 10), sharex=True)

# Plot equity curve on the first subplot
metrics.plot_equity(result, ax=ax1)
ax1.set_title(f"SPY SMA ({short_window}/{long_window}) Crossover: Equity Curve")

# Plot drawdown periods on the second subplot
metrics.plot_drawdown(result, ax=ax2)
ax2.set_title("Drawdown Periods")

plt.tight_layout() # Adjust layout to prevent overlapping titles/labels
plt.show()

### Next Steps

- Try different signals or strategies (e.g., momentum, mean reversion, volatility-based models).
- Implement parameter sweeps to optimize strategy parameters (like SMA windows).
- Test the framework with other assets (e.g., QQQ, EFA, IEF) or a portfolio of assets.
- Enhance the `metrics` module with additional performance measures (e.g., Sortino ratio, Calmar ratio, rolling Sharpe ratio, beta, alpha).
- Develop more sophisticated transaction cost models (e.g., including slippage, fixed costs).
- Add support for short selling or different position sizing schemes.